In [3]:
import requests
from skyfield.api import EarthSatellite, load
import pandas as pd

# 🚀 Step 1: Fetch TLE Data
TLE_URL = "https://www.celestrak.com/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
response = requests.get(TLE_URL)
tle_data = response.text.strip().split("\n")

# 🚀 Step 2: Parse TLE Data
ts = load.timescale()
satellite_names, inclinations, eccentricities, mean_motions = [], [], [], []
raan_list, arg_perigee_list, mean_anomaly_list = [], [], []

for i in range(0, len(tle_data), 3):  # TLEs are in sets of 3 lines
    try:
        name = tle_data[i].strip()
        line1 = tle_data[i + 1].strip()
        line2 = tle_data[i + 2].strip()

        satellite = EarthSatellite(line1, line2, name, ts)

        # Extract correct orbital parameters
        inclinations.append(satellite.model.inclo * (180 / 3.14159))  # Convert radians to degrees
        eccentricities.append(satellite.model.ecco)
        mean_motions.append(satellite.model.no_kozai * (1440 / (2 * 3.14159)))  # Convert rad/min to rev/day
        raan_list.append(satellite.model.nodeo * (180 / 3.14159))  # RAAN in degrees
        arg_perigee_list.append(satellite.model.argpo * (180 / 3.14159))  # Argument of Perigee
        mean_anomaly_list.append(satellite.model.mo * (180 / 3.14159))  # Mean Anomaly
        satellite_names.append(name)

    except Exception as e:
        print(f"⚠️ Error processing {name}: {e}")

# 🚀 Step 3: Save Data to CSV
df = pd.DataFrame({
    "Satellite Name": satellite_names,
    "Inclination (°)": inclinations,
    "Eccentricity": eccentricities,
    "Mean Motion (rev/day)": mean_motions,
    "RAAN (°)": raan_list,
    "Argument of Perigee (°)": arg_perigee_list,
    "Mean Anomaly (°)": mean_anomaly_list,
})

df.to_csv("tle_orbital_data.csv", index=False)
print("✅ TLE Data Saved to 'tle_orbital_data.csv'")


✅ TLE Data Saved to 'tle_orbital_data.csv'
